<a href="https://colab.research.google.com/github/hmyrcmn/GlobalAiProjects/blob/main/derin_ogrenme_odev_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import zipfile

# Google Drive'ı bağlayın
drive.mount('/content/drive')

Mounted at /content/drive


# data

In [32]:
# Zip dosyasının yolu
import os
zip_file_path = "/content/drive/MyDrive/maskData/dataset-20240508T180232Z-001.zip"

# Çıkartılacak hedef dizinin yolu
extract_dir = "/content/data4/"

# Zip dosyasını açın
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Tüm dosyaları belirtilen hedef dizine çıkartın
    zip_ref.extractall(extract_dir)

# Çıkartılan dosyaları gösterin
print("Zip dosyası başarıyla çıkartıldı. Çıkartılan dosyaların listesi:")
extracted_files = os.listdir(extract_dir)
for file_name in extracted_files:
    print(file_name)


Zip dosyası başarıyla çıkartıldı. Çıkartılan dosyaların listesi:
dataset


In [3]:
!git clone https://github.com/AlexeyAB/darknet.git


Cloning into 'darknet'...
remote: Enumerating objects: 15851, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15851 (delta 5), reused 13 (delta 4), pack-reused 15833
Receiving objects: 100% (15851/15851), 14.42 MiB | 8.46 MiB/s, done.
Resolving deltas: 100% (10671/10671), done.


In [27]:
import os

# Verilen dizindeki tüm dosyaları dolaşarak etiket dosyalarını oluşturma
def create_label_files(folder_path, class_name):
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            label_file_path = os.path.join(folder_path, filename.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg", ".txt"))
            with open(label_file_path, "w") as label_file:
                label_file.write(str(class_name))

# Train veri klasörlerinin yolu
train_folders = [
    "/content/data3/dataset/train/IncorrectMask",
    "/content/data3/dataset/train/WithMask",
    "/content/data3/dataset/train/WithoutMask"
]

# Validation veri klasörlerinin yolu
validation_folders = [
    "/content/data3/dataset/validation/IncorrectMask",
    "/content/data3/dataset/validation/WithMask",
    "/content/data3/dataset/validation/WithoutMask"
]

# Test veri klasörlerinin yolu
test_folders = [
    "/content/data3/dataset/test/IncorrectMask",
    "/content/data3/dataset/test/WithMask",
    "/content/data3/dataset/test/WithoutMask"
]

# Train verileri için etiket dosyalarını oluşturma
for folder in train_folders:
    create_label_files(folder, class_name=1 if "IncorrectMask" in folder else (0 if "WithMask" in folder else 2))

# Validation verileri için etiket dosyalarını oluşturma
for folder in validation_folders:
    create_label_files(folder, class_name=1 if "IncorrectMask" in folder else (0 if "WithMask" in folder else 2))

# Test verileri için etiket dosyalarını oluşturma
for folder in test_folders:
    create_label_files(folder, class_name=1 if "IncorrectMask" in folder else (0 if "WithMask" in folder else 2))


In [ ]:
def convert_labels_to_darknet_format(folder_path, num_classes):
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            label_file_path = os.path.join(folder_path, filename)
            with open(label_file_path, "r+") as label_file:
                lines = label_file.readlines()
                label_file.seek(0)
                label_file.truncate()
                for line in lines:
                    class_id = int(line.strip())  # Sınıf indeksi
                    x, y, w, h = 0.5, 0.5, 1.0, 1.0  # Normalleştirilmiş koordinatlar (örneğin, orta nokta ve tam görüntü)
                    label_file.write(f"{class_id} {x} {y} {w} {h}\n")
# Train veri klasörlerinin yolu
train_folders = [
    "/content/data3/dataset/train/IncorrectMask",
    "/content/data3/dataset/train/WithMask",
    "/content/data3/dataset/train/WithoutMask"
]

# Validation veri klasörlerinin yolu
validation_folders = [
    "/content/data3/dataset/validation/IncorrectMask",
    "/content/data3/dataset/validation/WithMask",
    "/content/data3/dataset/validation/WithoutMask"
]

# Test veri klasörlerinin yolu
test_folders = [
    "/content/data3/dataset/test/IncorrectMask",
    "/content/data3/dataset/test/WithMask",
    "/content/data3/dataset/test/WithoutMask"
]

# Veri setini Darknet formatına dönüştürme
for folders in [train_folders, validation_folders, test_folders]:
    for folder in folders:
        convert_labels_to_darknet_format(folder, num_classes=3)  # Toplam sınıf sayısı


# resize

In [28]:
import cv2
import os

# Tüm resimleri belirtilen boyuta yeniden boyutlandırma
def resize_images(folder_path, target_size):
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not read image {image_path}. Skipping...")
                continue
            resized_image = cv2.resize(image, target_size)
            cv2.imwrite(image_path, resized_image)

# Train, validation ve test veri klasörlerinin yolu
data_folders = [
    "/content/data3/dataset/train",
    "/content/data3/dataset/validation",
    "/content/data3/dataset/test"
]

# Tüm veri klasörleri için resimleri yeniden boyutlandırma
for folder in data_folders:
    for class_name in os.listdir(folder):
        class_path = os.path.join(folder, class_name)
        resize_images(class_path, (416, 416))


In [37]:
def convert_to_darknet_format(config_file_path, num_classes, batch_size, subdivisions, width, height):
    with open(config_file_path, 'r') as f:
        lines = f.readlines()

    # Genel parametreleri güncelle
    lines[1] = f'batch={batch_size}\n'
    lines[2] = f'subdivisions={subdivisions}\n'
    lines[5] = f'width={width}\n'
    lines[6] = f'height={height}\n'

    # Her bir YOLO katmanını güncelle
    yolo_count = 0
    for i, line in enumerate(lines):
        if line.startswith('[yolo]'):
            yolo_count += 1
            lines[i + 1] = f'classes={num_classes}\n'
            lines[i + 3] = f'filters={(num_classes + 5) * 3}\n'
            if yolo_count == 1:
                lines[i + 2] = f'anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401\n'
            elif yolo_count == 2:
                lines[i + 2] = f'anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401\n'
            else:
                lines[i + 2] = f'anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401\n'

    # Yapılandırma dosyasını kaydet
    with open(config_file_path, 'w') as f:
        f.writelines(lines)

# Örnek kullanım
convert_to_darknet_format(
    config_file_path='/content/darknet/cfg/yolov4.cfg',  # Yapılandırma dosyasının yolu
    num_classes=3,  # Toplam sınıf sayısı
    batch_size=64,  # Mini grup boyutu
    subdivisions=8,  # Alt bölümler sayısı
    width=416,  # Giriş genişliği
    height=416  # Giriş yüksekliği
)


# train


In [41]:
# !git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

# Darknet'i CUDA ile derleyin (GPU kullanımı için)
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make


/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: un

In [40]:
%cd darknet

./darknet detector train cfg/coco.data cfg/yolov4.cfg yolov4.conv.137 -map


SyntaxError: invalid syntax (<ipython-input-40-15400232a0ca>, line 3)

In [30]:
import os
import shutil
import xml.etree.ElementTree as ET

# Veri kümesinin ve alt klasörlerin yolları
data_dir = "/content/data3/dataset"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")
classes_file = os.path.join(data_dir, "classes.txt")

# Sınıfları (etiketleri) içeren bir liste oluşturma
classes = []
for file in os.listdir(data_dir):
    if file.endswith(".txt"):
        with open(os.path.join(data_dir, file), "r") as f:
            for line in f:
                class_name = line.strip()
                if class_name not in classes:
                    classes.append(class_name)

# Sınıfları classes.txt dosyasına yazma
with open(classes_file, "w") as f:
    for class_name in classes:
        f.write(class_name + "\n")

# Verileri train, val ve test klasörlerine ayırma
for class_name in classes:
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)

    if not os.path.exists(train_class_dir):
        os.makedirs(train_class_dir)
    if not os.path.exists(val_class_dir):
        os.makedirs(val_class_dir)
    if not os.path.exists(test_class_dir):
        os.makedirs(test_class_dir)

    for file in os.listdir(data_dir):
        if file.endswith(".jpg"):
            image_path = os.path.join(data_dir, file)
            image_name = file.split(".")[0]

            # Etiket dosyasını bulma
            label_file_path = os.path.join(data_dir, image_name + ".txt")
            if not os.path.exists(label_file_path):
                continue

            # Etiket dosyasını açma ve sınıfları okuma
            with open(label_file_path, "r") as f:
                class_labels = [line.strip() for line in f]

            # Görüntüleri train, val veya test klasörlerine taşıma
            if len(class_labels) > 0:
                if random.random() < 0.8:  # %80 train, %20 val
                    dest_dir = train_class_dir
                elif random.random() < 0.9:  # %10 val, %10 test
                    dest_dir = val_class_dir
                else:
                    dest_dir = test_class_dir

                shutil.copy2(image_path, dest_dir)
                shutil.copy2(label_file_path, dest_dir)

print(classes)

[]


In [ ]:
;darknet.exe train \
    -data cfg/coco.data \
    -cfg cfg/yolov4.cfg \
    -weights yolov4.weights


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2

!git clone https://github.com/ultralytics/yolov5


model = torch.hub.load("ultralytics/yolov5", "yolov5s")


Cloning into 'yolov5'...
remote: Enumerating objects: 16582, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 16582 (delta 30), reused 39 (delta 18), pack-reused 16522
Receiving objects: 100% (16582/16582), 15.06 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (11389/11389), done.


/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 203.3 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 9.4s, installed 2 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-5-10 Python-3.10.12 torch-2.2.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 187MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [5]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s")
img = "https://cdn.teyit.org/wp-content/uploads/2017/07/muslum-gursesin-tupac-ve-snoop-dogg-ile-fotografi-oldugu-iddiasi.jpg"


result = model(img)
result.print()



Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-10 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7e117309c270>

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,8))
plt.imshow(np.squeeze(result.render()))
plt.show()

In [ ]:

!cp "/kaggle/input/yamlfordataset/dataset.yml" "/kaggle/working/yolov5"

!cd yolov5 && python train.py --img 640 --batch 16 --epochs 100 --data dataset.yml --weights yolov5s.pt --workers 2

results = "yolov5/runs/train/exp/results.png"
results = cv2.imread(results)
plt.figure(figsize=(14, 10))
plt.imshow(results)


conf_mat = "yolov5/runs/train/exp/confusion_matrix.png"
conf_mat = cv2.imread(conf_mat)
plt.figure(figsize=(12, 10))
plt.imshow(conf_mat)

f1_curve = "yolov5/runs/train/exp/F1_curve.png"
f1_curve = cv2.imread(f1_curve)
plt.figure(figsize=(10, 10))
plt.imshow(f1_curve)

# Yeni Bölüm

In [ ]:
# I import libraries that I will use.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# For preparing
import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps

In [ ]:
with open('/content/face-mask-detection/annotations/maksssksksss0.xml') as f:
    contents = f.read()
    print(contents)

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss1.png")

In [ ]:
# creating a image1 object
im1 = Image.open(r"/content/face-mask-detection/images/maksssksksss100.png")

# applying grayscale method
im2 = ImageOps.grayscale(im1)

im2

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h

    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h

    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center, width, height
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2

    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)

    return [xmin, ymin, xmax, ymax]

classes = []

input_dir = "/content/face-mask-detection/annotations"
output_dir = "/content/labels"
image_dir = "/content/face-mask-detection/images"

os.mkdir(output_dir)

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

import glob

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # Parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text

        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)

        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a yolo format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))
# generate the classes file as reference
with open("/content/classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))